# Исследование по метрикам на основе данных моего произношения

## Импорт библиотеки

In [2]:
# Импорт библиотеки
import sys
import os
import re
import numpy as np
from jiwer import wer, cer, mer, wil

import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Функция для подсчета расстояния Левенштейна
def levenshtein_distance(a, b):
    m, n = len(a), len(b)
    dp = np.zeros((m + 1, n + 1), dtype=int)
    
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
        
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,    # удаление
                               dp[i][j - 1] + 1,    # вставка
                               dp[i - 1][j - 1] + 1)  # замена
    return dp[m][n]

# Тексты
reference = "Искусственный интеллект (ИИ) - это область компьютерных наук, создающая системы для выполнения задач, требующих человеческого интеллекта: распознавание речи, обработка изображений, принятие решений. Искусственный интеллект проникает во все сферы жизни, от рекомендаций в онлайн-магазинах до медицинских диагнозов."
recognized = "Рвотчины технари мнящие мир и проносящие на четверни двигатель барылочных опрашивает однако я выражаю это и вызывают Аспектами будущими исследования и разработки в вопросе Проект путь играть ключевую роль в формировании общества и экономики"

# Разделение на слова
ref_words = reference.split()
rec_words = recognized.split()

# Вычисление MER для каждого слова
mer_per_word = []

for ref_word in ref_words:
    min_distance = float('inf')
    for rec_word in rec_words:
        distance = levenshtein_distance(ref_word, rec_word)
        if distance < min_distance:
            min_distance = distance
    mer_per_word.append(min_distance / len(ref_word))

In [6]:
mer_per_word

[0.6923076923076923,
 0.6666666666666666,
 1.0,
 1.0,
 0.0,
 0.5714285714285714,
 0.75,
 0.6,
 0.7777777777777778,
 0.8571428571428571,
 0.6666666666666666,
 0.7,
 0.8333333333333334,
 0.6666666666666666,
 0.6923076923076923,
 0.7272727272727273,
 0.6153846153846154,
 0.8,
 0.4444444444444444,
 0.75,
 0.5,
 0.75,
 0.6923076923076923,
 0.6666666666666666,
 0.4444444444444444,
 0.5,
 0.6666666666666666,
 0.8,
 0.8333333333333334,
 1.0,
 0.6666666666666666,
 0.0,
 0.8125,
 1.0,
 0.7272727272727273,
 0.8]

In [9]:
# Разделение на слова
ref_words = reference.split()
rec_words = recognized.split()

# Функция для вычисления MER по каждому слову
def calculate_mer_per_word(ref_words, rec_words):
    mer_per_word = []
    for ref_word in ref_words:
        ref_text = ref_word
        rec_text = ' '.join(rec_words)
        word_mer = mer([ref_text], [rec_text])
        mer_per_word.append(word_mer)
    return mer_per_word

# Вычисление MER по каждому слову
mer_per_word = calculate_mer_per_word(ref_words, rec_words)

In [8]:
print(mer_per_word)

[1.0, 1.0, 1.0, 1.0, 0.9705882352941176, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9705882352941176, 1.0, 1.0, 1.0, 1.0]


In [41]:
# Сбор список слов
from sentences_text import sentences_text

## Коллекций слова

In [42]:
for theme, words in sentences_text.items():
    print(f"Тема: {theme}")
    print("Предложение:", words)
    print("\n")

Тема: Искусственный интеллект
Предложение: Искусственный интеллект (ИИ) - это область компьютерных наук, создающая системы для выполнения задач, требующих человеческого интеллекта: распознавание речи, обработка изображений, принятие решений. Искусственный интеллект проникает во все сферы жизни, от рекомендаций в онлайн-магазинах до медицинских диагнозов.

Главное преимущество ИИ - способность обрабатывать большие объемы данных и выявлять закономерности. В медицине он помогает диагностировать заболевания, анализируя снимки и историю болезни. В бизнесе искусственный интеллект используется для прогнозирования рынка, оптимизации цепочек поставок и автоматизации процессов.

ИИ базируется на алгоритмах машинного обучения, позволяющих системам обучаться на основе данных. Современные ИИ-системы используют глубокие нейронные сети, имитирующие работу мозга. Для разработки таких моделей требуются мощные вычислительные ресурсы и инструменты, такие как TensorFlow и PyTorch.

Искусственный интеллект

## Подключение к БД

In [43]:
# Соединение с БД
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="asr_text",
    user="postgres",
    password="postgres"
)
cur = conn.cursor()

In [44]:
# Получение данных из таблицы
cur.execute("SELECT * FROM audio_records")
data = cur.fetchall()

In [45]:
# Закрытие соединения
cur.close()
conn.close()

## Создание и анализ датафрейма

In [46]:
# Создание DataFrame из полученных данных
df = pd.DataFrame(data, columns=['id', 'topic', 'paragraph_text', 'transcript_text_yandex', 'transcript_text_salutespeech', 'transcript_text_mbart50', 'voice_recording', 'record_date'])

In [47]:
# Преобразование столбца record_date в формат datetime
df['record_date'] = pd.to_datetime(df['record_date'])

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            43 non-null     int64         
 1   topic                         43 non-null     object        
 2   paragraph_text                43 non-null     object        
 3   transcript_text_yandex        43 non-null     object        
 4   transcript_text_salutespeech  43 non-null     object        
 5   transcript_text_mbart50       43 non-null     object        
 6   voice_recording               43 non-null     object        
 7   record_date                   43 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.8+ KB


In [57]:
df.head(5)

,id,topic,paragraph_text,transcript_text_yandex,transcript_text_salutespeech,transcript_text_mbart50,voice_recording,record_date
0,5,Искусственный интеллект,Искусственный интеллект - революционная технол...,Рвотчины технари мнящие мир и проносящие на че...,"Эффект революционный техногии, меняющий мир, п...","они киллект, революция, тихонги, мне очень яны...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-15
1,13,Веб-сокет,Веб-сокеты - мощный инструмент для разработки ...,Веб секрет мощный интернет для разработки веб ...,Рен постояние 3 направлений сидни и низкая за...,"ведь все-таки, тёмненький интеллигент, проживш...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
2,14,Плавание,"Плавание - это вид спорта и активного отдыха, ...",Прямо на этой битве спорят и активисты выходят...,"Про мне это вид спорт и активный. Выходи, закл...","про мне эти видосы, и активные охоты и закруча...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
3,23,Технологическая сингулярность,"Технологическая сингулярность - это теория, пр...",Процессорывающий момент когда интеллект провод...,Проводит человеческий интеллект и технологии ...,"— из-вторых, неверность этот яры, протягивающи...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
4,38,Плавание,Существует несколько стилей плавания с разными...,Как ровными техническими осипшими крыла на гру...,"Считывание, сколько стилей прямо сравнимы техн...","— ты говорю, нет, как ощутили, я промолленными...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-17


In [50]:
# Экспорт DataFrame в CSV файл
df.to_csv('database.csv', index=False)

##### Поскольку модель выдавала текст с знаками и другие, поэтому будем устранять их и заново рассчитать метрику

## Предобработка данных

### Изменение индеса в временный ряд

In [54]:
# Установим индекс по столбцу 'record_date'
research_df = df.set_index('record_date')

# Отбросим ненужные столбцы
# research_df = research_df.drop(['id'], axis=1)

# Убедимся, что индекс отсортирован
research_df = research_df.sort_index()

In [55]:
research_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43 entries, 2024-06-15 to 2024-06-17
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            43 non-null     int64 
 1   topic                         43 non-null     object
 2   paragraph_text                43 non-null     object
 3   transcript_text_yandex        43 non-null     object
 4   transcript_text_salutespeech  43 non-null     object
 5   transcript_text_mbart50       43 non-null     object
 6   voice_recording               43 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.7+ KB


In [56]:
research_df.head()

,id,topic,paragraph_text,transcript_text_yandex,transcript_text_salutespeech,transcript_text_mbart50,voice_recording
record_date,,,,,,,
2024-06-15,5,Искусственный интеллект,Искусственный интеллект - революционная технол...,Рвотчины технари мнящие мир и проносящие на че...,"Эффект революционный техногии, меняющий мир, п...","они киллект, революция, тихонги, мне очень яны...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\..."
2024-06-15,1,Искусственный интеллект,Искусственный интеллект (ИИ) - это область ком...,Искусственный интеллект это вопрос компьютерны...,,"едивствительный характер, а то я, попросту, ко...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\..."
2024-06-15,4,Искусственный интеллект,ИИ базируется на алгоритмах машинного обучения...,Секс интеллект позируется на алгоритмах машина...,Интеллект позируется на алгоритмах машины появ...,"здесь к примеру интеллигент, пояснивая причину...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\..."
2024-06-15,2,Искусственный интеллект,Главное преимущество ИИ - способность обрабаты...,Структурный тракт Трапающий тракт Так из за по...,Осуществлённый тракт от оплаты компьютерных но...,"здесь копустимный тракт, а то просит купечески...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\..."
2024-06-16,26,Технологическая сингулярность,Технологическая сингулярность представляет зах...,Технологическая сингулярность представляет зах...,Технологической сингулярности представляет зах...,"""техноногие чиновники, представляя и прерывающ...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\..."


### Разбиение на 4 частей

In [36]:
# Функция для разбиения текста на 4 части
def split_paragraph(paragraph):
    parts = paragraph.split('\n\n')
    if len(parts) < 4:
        parts.extend([''] * (4 - len(parts)))
    return parts

In [37]:
# Применим функцию ко всем строкам в столбце 'paragraph_text'
df[['part1', 'part2', 'part3', 'part4']] = pd.DataFrame(df['paragraph_text'].apply(split_paragraph).tolist(), index=df.index)

In [38]:
# Создадим новый DataFrame с разбитыми параграфами
parts_df = df.melt(id_vars=['topic', 'transcript_text_yandex', 'transcript_text_salutespeech', 'transcript_text_mbart50'], 
                   value_vars=['part1', 'part2', 'part3', 'part4'],
                   var_name='part', value_name='paragraph_part').sort_values(by=['topic', 'part'])

# Удалим пустые строки
parts_df = parts_df[parts_df['paragraph_part'] != '']

In [39]:
parts_df

,topic,transcript_text_yandex,transcript_text_salutespeech,transcript_text_mbart50,part,paragraph_part
16,ВВП к ППС,Вы собираетесь тронь с учетом различий до уров...,В концепте в. Позволяет сравнивать экономическ...,"в принципе, в эпике, по песе, по песе, с бюлле...",part1,"Концепция ""ВВП к ППС"" позволяет сравнивать эко..."
17,ВВП к ППС,Основные Корректировать Имущественным это позв...,Основные преимущества возможность корректирова...,"обивные приспособы, тупые, промышленность, кор...",part1,"Концепция ""ВВП к ППС"" позволяет сравнивать эко..."
18,ВВП к ППС,Включи товаров в оборонных странах потирания к...,"Расчет включить, сравните на широкий набор тов...",ракурсе бы я по-попески: включи отец русни нит...,part1,"Концепция ""ВВП к ППС"" позволяет сравнивать эко..."
19,ВВП к ППС,Машина Страны И урамных тин использование этог...,Учитывающие ранее значение Бо читаемость жизн...,в библии по попечень можно не тромить для себя...,part1,"Концепция ""ВВП к ППС"" позволяет сравнивать эко..."
20,ВВП к ППС,Боемышечность корректировать роеволичивость Эт...,I Ровно нативность страны это позволяет объек...,"объявленный прем, чтобы поэтапость, боеспособн...",part1,"Концепция ""ВВП к ППС"" позволяет сравнивать эко..."
...,...,...,...,...,...,...
156,Технологическая сингулярность,Технологическая сингулярность представляет зах...,Технологической сингулярности представляет зах...,"""техноногие чиновники, представляя и прерывающ...",part4,Технологическая сингулярность представляет зах...
172,Технологическая сингулярность,Технологическая сингулярность в этот европе пр...,"Технологическая сингулярность в теории, протис...","их нагочеки, сеньоры, протягивающие метод, ког...",part4,Технологическая сингулярность представляет зах...
173,Технологическая сингулярность,Прогрессив вопросительный спектр ключей группо...,Прогресс вопросы котлет включи группа почини и...,"прогресс был попроси к в интеллекте, включая г...",part4,Технологическая сингулярность представляет зах...
174,Технологическая сингулярность,Сингулярность вызывает множество философских И...,"Сингулярность вызывает наши этические вопросы,...",сигнаг уезжает взвешенной шепелявой ощечке и в...,part4,Технологическая сингулярность представляет зах...


### Установка диапазона времени дат

In [25]:
# Фильтруем данные по диапазону дат
start_date = '2024-04-22'
end_date = '2024-05-17'

research_df = research_df.loc[start_date:end_date]

### Удаление лишних знаков в тексте

In [26]:
def clean_text(text):
    # Удаление всех знаков пунктуации
    return re.sub(r'[^\w\s]', '', text)

In [27]:
research_df['transcription_word'] = research_df['transcription_word'].apply(clean_text)

In [28]:
research_df.head()

### Установка метрик

In [29]:
def iwer(reference_sentence, hypothesis_sentence) -> float:
    """
    Вычисляет Inflectional Word Error Rate (IWER) между предложением-эталоном и гипотезой.

    Параметры:
    reference_sentence (str): Предложение-эталон (правильный вариант).
    hypothesis_sentence (str): Гипотеза (предсказанный вариант).

    Возвращает:
    float: Значение Inflectional Word Error Rate (IWER) в процентах.
    """
    reference_words = reference_sentence.split()
    hypothesis_words = hypothesis_sentence.split()

    # Находим количество слов в предложении-эталоне
    total_words = len(reference_words)

    # Считаем количество неправильно распознанных слов
    incorrect_words = sum(1 for ref, hyp in zip(reference_words, hypothesis_words) if ref != hyp)

    # Вычисляем Inflectional Word Error Rate (IWER)
    iwer_score = incorrect_words / total_words

    return iwer_score

In [30]:
def calculate_metrics(df):
    df['wer'] = df.apply(lambda row: wer(row['word_for_check'], row['transcription_word']), axis=1)
    df['cer'] = df.apply(lambda row: cer(row['word_for_check'], row['transcription_word']), axis=1)
    df['mer'] = df.apply(lambda row: mer(row['word_for_check'], row['transcription_word']), axis=1)
    df['wil'] = df.apply(lambda row: wil(row['word_for_check'], row['transcription_word']), axis=1)
    df['iwer'] = df.apply(lambda row: iwer(row['word_for_check'], row['transcription_word']), axis=1)

    return df

In [31]:
# Вычисление метрик
research_df = calculate_metrics(research_df)

In [32]:
research_df.head(10)

In [33]:
research_df[research_df['word_for_check'] == 'робот'].head

## Анализ данных

In [89]:
# Сохраняем метрики
metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']

# Словарь цветов для метрик
colors = {
    'wer': 'blue',
    'cer': 'orange',
    'mer': 'green',
    'wil': 'black',
    'iwer': 'purple'
}

### Функция отрисовки графики

In [90]:
def draw_plot(word):
    # Фильтрация данных по слову
    filtered_df = research_df[research_df['word_for_check'] == word].sort_index()

    for metric in metrics:
        # Построение графика
        plt.figure(figsize=(15, 5))
        plt.plot(filtered_df.index, filtered_df[metric], marker='o', linestyle='-', color=colors[metric], label=metric.upper())
        
        # Скользящее среднее
        rolling_mean = filtered_df[metric].rolling(window=3).mean()
        plt.plot(filtered_df.index, rolling_mean, color='red', linestyle='--', label=f'{metric.upper()} (3-дневное скользящее среднее)')
        
        plt.title(f'Динамика {metric.upper()} для слова "{word}"', fontweight='bold', fontsize=12)
        plt.xlabel('Дата', fontweight='bold', fontsize=12)
        plt.ylabel(metric.upper(), fontweight='bold', fontsize=12)
        plt.grid(True)
        plt.xticks(rotation=45)
        
        # Добавление надписей
        for idx, row in filtered_df.iterrows():
            plt.annotate(row['transcription_word'], (idx, row[metric]), textcoords="offset points", xytext=(0,10), ha='center', rotation=45)
        
        plt.legend()
        plt.tight_layout()
        
        # Показ графика
        plt.show()

### Слово "Робот"

In [91]:
draw_plot('робот')

### Слово "Инженер"

In [60]:
draw_plot('инженер')

In [107]:
# Функция для вычисления изменений метрик
def compute_metrics_change(group):
    metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']
    changes = {metric: (group[metric].iloc[-1] - group[metric].iloc[0]) for metric in metrics}
    
    return changes

In [108]:
# Функция для построения графиков изменения метрик по темам
def draw_change_plot_by_theme(collection):
    metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']

    for theme, words in collection.items():
        # Фильтрация данных по словам из текущей темы
        theme_df = research_df[research_df['word_for_check'].isin(words)]

        if theme_df.empty:
            continue

        # Группировка по слову и вычисление изменений метрик
        grouped_df = theme_df.groupby('word_for_check')
        changes = {word: compute_metrics_change(group) for word, group in grouped_df}

        for metric in metrics:
            # Создание графика для текущей метрики
            plt.figure(figsize=(15, 5))
            words = list(changes.keys())
            values = [changes[word][metric] for word in words]

            # Отфильтруем слова с некорректными значениями
            words = [word for word, value in zip(words, values) if pd.notnull(value) and np.isfinite(value)]
            values = [value for value in values if pd.notnull(value) and np.isfinite(value)]

            # Определение направления изменения
            arrows = ['↓' if val < 0 else '↑' for val in values]
            labels = [f'{val:.2f} {arrow}' for val, arrow in zip(values, arrows)]
            colors = ['green' if val < 0 else 'red' for val in values]

            # Построение столбчатой диаграммы
            plt.bar(words, values, color=colors)

            # Добавление подписей
            for i, (word, label) in enumerate(zip(words, labels)):
                plt.text(i, values[i], label, ha='center', va='bottom' if values[i] < 0 else 'top')

            plt.title(f'Изменение {metric.upper()} по теме "{theme}"', fontweight='bold', fontsize=12)
            plt.xlabel('Слово', fontweight='bold', fontsize=12)
            plt.ylabel(f'Изменение {metric.upper()}', fontweight='bold', fontsize=12)
            plt.grid(True)
            plt.xticks(rotation=45)
            
            plt.tight_layout()
            plt.show()

In [109]:
# Пример вызова функции
draw_change_plot_by_theme(collection)